# Event log creation - version 2

Following feedback from statisticians, this script (which should be run after step 6) will revise certain fields to conform with subsequent survival analysis and regression analysis.

In [ ]:
suppressPackageStartupMessages(library(tidyverse))
library(lubridate)
library(glue)
suppressPackageStartupMessages(library(bupaR))
library(processanimateR)
library(ggplot2)
library(ggfortify)
# # Debian / Ubuntu sudo apt-get install libv8-dev using a terminal window first before gtsummary
library(gtsummary)
library(networkD3)
suppressPackageStartupMessages(library(flextable))

In [ ]:
imd_ethnicity_fn <- function(df) {
 df %>%
    mutate(
        imd_quintile = as.factor(case_when(
            as.integer(imd_decile) %in% c(1, 2) ~ 1,
            as.integer(imd_decile) %in% c(3, 4) ~ 2,
            as.integer(imd_decile) %in% c(5, 6) ~ 3,
            as.integer(imd_decile) %in% c(7, 8) ~ 4,
            as.integer(imd_decile) %in% c(9, 10) ~ 5,
            TRUE ~ NA_integer_
        )),
        ethnicity_simple = fct_relevel(str_extract(ethnicity, "[^:]+"), 'White')
    ) %>%
    filter( age <= 107)
}

In [ ]:
#summary_df1 <- readRDS(glue::glue("output/grouping1_summary_df.rds"))

In [ ]:
summary_df1 <- readRDS(glue::glue("output/grouping1_summary_df.rds")) %>%
    imd_ethnicity_fn()
summary_df2 <- readRDS(glue::glue("output/grouping2_summary_df.rds")) %>%
    imd_ethnicity_fn()
summary_df3 <- readRDS(glue::glue("output/grouping3_summary_df.rds")) %>%
    imd_ethnicity_fn()

In [ ]:
summary_df1 %>% count(strata)

In [ ]:
summarise_data_fn <- function(summary_df) {
    summary_df %>% select(
        strata, 
        ooh, 
        age, 
        sex, 
        imd_quintile, 
        ethnicity_simple, 
        hosp, 
        first_next, 
        non_avoid_adn, 
        sum_contacts_ED,
        sum_contacts_CAD,
        sum_contacts_AC, 
        sum_contacts_GP, 
        sum_contacts_IP, 
        sum_contacts_IUC
    ) %>%
    #glimpse()
    tbl_summary(
     by = "strata",
     type = list(starts_with("sum_contacts_") ~ "continuous", non_avoid_adn ~ 'continuous'),
     statistic = list(
         ooh ~ "{n} ({p}%)",
         age ~ "{median} ({p25}, {p75})",
         sex ~ "{n} ({p}%)",
         imd_quintile ~ "{n} ({p}%)",
         ethnicity_simple ~  "{n} ({p}%)",
         non_avoid_adn ~ "{sum}",
         hosp ~ "{n} ({p}%)",
         first_next ~ "{n} ({p}%)",
         sum_contacts_CAD ~ "{sum}",
         sum_contacts_AC  ~ "{sum}",
         sum_contacts_ED  ~ "{sum}",
         sum_contacts_GP  ~ "{sum}",
         sum_contacts_IP  ~ "{sum}",
         sum_contacts_IUC ~ "{sum}"
     ),
     label = list(
         ooh ~ "Out-of-hours",
         age ~ "Age in years",
         sex ~ "Sex",
         imd_quintile ~ "IMD quintile",
         ethnicity_simple ~ "Ethnicity",
         non_avoid_adn ~ "Non-avoidable ED attendances",
         hosp ~ "Hospital ED attendance",
         first_next ~ "First healthcare contact after index call",
         sum_contacts_CAD ~ "999 ambulance calls",
         sum_contacts_AC ~ "Abandoned calls",
         sum_contacts_ED ~ "ED attendances",
         sum_contacts_GP ~ "GP contacts",
         sum_contacts_IP ~ "In-patient admissions",
         sum_contacts_IUC ~ "Triaged 111 calls"
     )
    ) %>% bold_labels() %>% add_stat_label() %>%
    modify_header(
        update = list(
          stat_1 ~ "**Abandoned Call** (n={n})",
          stat_2 ~ "**Triaged 111 Call** (n={n})"
        ))
}

In [ ]:
st1 <- summarise_data_fn(summary_df1)
st2 <- summarise_data_fn(summary_df2)
st3 <- summarise_data_fn(summary_df3)
# st4 <- summarise_data_fn(summary_df4)

In [ ]:
st_merged <- tbl_merge(list(st1, st2, st3), tab_spanner = c('Grouping 1', 'Grouping 2', 'Grouping 3'))

In [ ]:
st_merged %>% as_tibble()

In [ ]:
saveRDS(st_merged, 'output/st_merged_table_one.rds')

In [ ]:
st_merged %>%
    as_gt() %>%
    gt::gtsave(filename = glue::glue("output/table1_all_cohorts.html"))

In [ ]:
st_merged %>%
    as_flex_table() %>%
    save_as_docx(path = glue::glue("output/table1_all_cohorts.docx"))